In [ ]:
from google.colab import drive 
drive.mount('/mntDrive')

Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [ ]:
! rm -r ocrpostcorrection

In [ ]:
!git clone https://github.com/jvdzwaan/ocrpostcorrection.git

Cloning into 'ocrpostcorrection'...
remote: Enumerating objects: 723, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 723 (delta 88), reused 92 (delta 48), pack-reused 588
Receiving objects: 100% (723/723), 1.18 MiB | 12.81 MiB/s, done.
Resolving deltas: 100% (453/453), done.


In [ ]:
!pip install ./ocrpostcorrection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ocrpostcorrection
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 451 kB 12.3 MB/s 
     |████████████████████████████████| 325 kB 60.1 MB/s 
     |████████████████████████████████| 58 kB 6.8 MB/s 
     |████████████████████████████████| 5.5 MB 67.3 MB/s 
     |████████████████████████████████| 115 kB 72.7 MB/s 
     |████████████████████████████████| 212 kB 73.2 MB/s 
     |████████████████████████████████| 182 kB 72.1 MB/s 
     |████████████████████████████████| 127 kB 72.4 MB/s 
     |██████

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading data

In [ ]:
data_base_dir = Path('/Users/janneke/Documents/Documents – Janneke’s MacBook/data/ocrpostcorrection')

In [ ]:
data_base_dir = Path('/mntDrive/MyDrive/data/ocrpostcorrection')

In [ ]:
!ls /mntDrive/MyDrive/data/

ocrpostcorrection


In [ ]:
in_file = data_base_dir/'icdar-task2-dataset-20221031'/'task2dataset-no-duplicates.csv'
data = pd.read_csv(in_file, index_col=0)
data = data.fillna('')

In [ ]:
train = data.query('dataset == "train"')
val = data.query('dataset == "val"')

In [ ]:
from ocrpostcorrection.error_correction import generate_vocabs, get_text_transform

vocab_transform = generate_vocabs(train)
text_transform = get_text_transform(vocab_transform)

In [ ]:
from torch.utils.data import DataLoader

from ocrpostcorrection.error_correction import SimpleCorrectionDataset, collate_fn

max_len = 22
batch_size = 256

train_dataset = SimpleCorrectionDataset(train, max_len=max_len)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, collate_fn=collate_fn(text_transform))

val_dataset = SimpleCorrectionDataset(val, max_len=max_len)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn(text_transform))

In [ ]:
print('num train samples', len(train_dataset))
print('num val samples', len(val_dataset))

num train samples 744738
num val samples 103272


In [ ]:
from ocrpostcorrection.error_correction import validate_model

In [ ]:
from tqdm.notebook import tqdm

class StopExecution(Exception):
    def _render_traceback_(self):
        return []

def train_model(train_dl, val_dl, model=None, optimizer=None, num_epochs=5, valid_niter=5000, 
                model_save_path='model.rar', max_num_patience=5, max_num_trial=5, 
                lr_decay=0.5, device='cpu'):
    num_iter = 0
    report_loss = 0
    report_examples = 0
    val_loss_hist = []
    num_trial = 0
    patience = 0

    model.train()

    for epoch in range(1, num_epochs+1):
        cum_loss = 0
        cum_examples = 0

        for src, tgt in tqdm(train_dl):
            #print(f'src: {src.size()}; tgt: {tgt.size()}')
            num_iter += 1

            batch_size = src.size(1)

            src = src.to(device)
            tgt = tgt.to(device)
            encoder_hidden = model.encoder.initHidden(batch_size=batch_size, device=device)

            # print(input_hidden.size())

            example_losses, _ = model(src, encoder_hidden, tgt)
            example_losses = -example_losses
            batch_loss = example_losses.sum()
            loss = batch_loss / batch_size

            bl = batch_loss.item()
            report_loss += bl
            report_examples += batch_size

            cum_loss += bl
            cum_examples += batch_size

            optimizer.zero_grad()
            loss.backward()

            # clip gradient
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)

            optimizer.step()

            if num_iter % valid_niter == 0:
                val_loss = validate_model(model, val_dl, device)
                print(f'Epoch {epoch}, iter {num_iter}, avg. train loss {report_loss/report_examples}, avg. val loss {val_loss}')

                report_loss = 0
                report_examples = 0

                better_model = len(val_loss_hist) == 0 or val_loss < min(val_loss_hist)
                if better_model:
                    print(f'Saving model and optimizer to {model_save_path}')
                    torch.save({
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      }, model_save_path)
                elif patience < max_num_patience:
                    patience += 1
                    print(f'hit patience {patience}')

                    if patience == max_num_patience:
                        num_trial += 1
                        print(f'hit #{num_trial} trial')
                        if num_trial == max_num_trial:
                            print('early stop!')
                            raise StopExecution('early stop!')

                        # decay lr, and restore from previously best checkpoint
                        lr = optimizer.param_groups[0]['lr'] * lr_decay
                        print(f'load previously best model and decay learning rate to {lr}')

                        # load model
                        checkpoint = torch.load(model_save_path)
                        model.load_state_dict(checkpoint['model_state_dict'])
                        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                        
                        model = model.to(device)
                        
                        # set new lr
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr

                        # reset patience
                        patience = 0
                    

                val_loss_hist.append(val_loss)



In [ ]:
out_dir = data_base_dir/'results'/'simple_correction_model'
out_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
from ocrpostcorrection.error_correction import SimpleCorrectionSeq2seq

hidden_size = 256
dropout = 0.1
model = SimpleCorrectionSeq2seq(len(vocab_transform['ocr']), 
                                hidden_size, 
                                len(vocab_transform['gs']), 
                                dropout, 
                                max_len, 
                                teacher_forcing_ratio=0.5,
                                device=device)
model.to(device)    
optimizer = torch.optim.Adam(model.parameters())

msp = out_dir/'model.rar'

train_model(train_dl=train_dataloader, 
            val_dl=val_dataloader,
            model=model, 
            optimizer=optimizer, 
            model_save_path=msp, 
            num_epochs=25, 
            valid_niter=1000, 
            max_num_patience=5, 
            max_num_trial=5, 
            lr_decay=0.5, 
            device=device)

  0%|          | 0/2910 [00:00<?, ?it/s]

Epoch 1, iter 1000, avg. train loss 22.811994752883912, avg. val loss 15.614160314168833
Saving model and optimizer to /mntDrive/MyDrive/data/ocrpostcorrection/results/simple_correction_model/model.rar
Epoch 1, iter 2000, avg. train loss 15.882390301704406, avg. val loss 13.397783801136198
Saving model and optimizer to /mntDrive/MyDrive/data/ocrpostcorrection/results/simple_correction_model/model.rar


  0%|          | 0/2910 [00:00<?, ?it/s]

Epoch 2, iter 3000, avg. train loss 14.126266974162874, avg. val loss 11.960924853491038
Saving model and optimizer to /mntDrive/MyDrive/data/ocrpostcorrection/results/simple_correction_model/model.rar
Epoch 2, iter 4000, avg. train loss 13.333274574279786, avg. val loss 11.695720509783175
Saving model and optimizer to /mntDrive/MyDrive/data/ocrpostcorrection/results/simple_correction_model/model.rar
Epoch 2, iter 5000, avg. train loss 12.7490910654068, avg. val loss 11.30207309815985
Saving model and optimizer to /mntDrive/MyDrive/data/ocrpostcorrection/results/simple_correction_model/model.rar


  0%|          | 0/2910 [00:00<?, ?it/s]

...

  0%|          | 0/2910 [00:00<?, ?it/s]

Epoch 15, iter 41000, avg. train loss 9.74282352486376, avg. val loss 8.878883864177965
hit patience 1
Epoch 15, iter 42000, avg. train loss 9.74165991306305, avg. val loss 8.942062592467623
hit patience 2
Epoch 15, iter 43000, avg. train loss 9.585915051460265, avg. val loss 8.79634273433604
hit patience 3


  0%|          | 0/2910 [00:00<?, ?it/s]

Epoch 16, iter 44000, avg. train loss 9.77400199716477, avg. val loss 8.9867198452664
hit patience 4
Epoch 16, iter 45000, avg. train loss 9.599046401023864, avg. val loss 8.961717552842856
hit patience 5
hit #5 trial
early stop!


StopExecution: ignored